In [61]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE


# Computes mae in batches so that we don't have memory issue in big dataset
def compute_MAE(sess, train_sparse, output_layer, bsize):

    loss_mae = 0
    tot_users = train_sparse.shape[0]
    train_sparse = train_sparse.tocsr()
    # print(tot_users)

    for i in range(int(tot_users/bsize)+1):

        to = (i+1)*bsize
        if to > tot_users:
            to = tot_users
            
        epoch_x = train_sparse[ i*bsize : to ]
        epoch_x = epoch_x.toarray()

        output_train = sess.run(output_layer, feed_dict={X:epoch_x})
        loss_mae += np.sum(abs(output_train - epoch_x))

    mae = loss_mae / (train_sparse.shape[0]*train_sparse.shape[1])
    return mae


def compute_RMSE(sess, train_sparse, output_layer, bsize):

    loss_rmse = 0
    tot_users = train_sparse.shape[0]
    train_sparse = train_sparse.tocsr()
    # print(tot_users)

    for i in range(int(tot_users/bsize)+1):

        to = (i+1)*bsize
        if to > tot_users:
            to = tot_users
            
        epoch_x = train_sparse[ i*bsize : to ]
        epoch_x = epoch_x.toarray()

        output_train = sess.run(output_layer, feed_dict={X:epoch_x})
        loss_rmse += np.sum(np.square(output_train - epoch_x))

    rmse = loss_rmse / (train_sparse.shape[0]*train_sparse.shape[1])
    return rmse

In [64]:
ratings_df = pd.read_csv('../datasets/ml-latest-small/ratings.csv', sep=",")
ratings_df = ratings_df.drop('timestamp', axis=1)
# ratings_df = ratings_df.head(len(ratings_df)//100)

In [65]:
r = ratings_df['rating'].values.astype(float)
min_max_scaler = sk.preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(r.reshape(-1,1))
df_normalized = pd.DataFrame(x_scaled)
ratings_df['rating'] = df_normalized

# Preprocessing
np_users = ratings_df.userId.values
np_items = ratings_df.movieId.values

unique_users = np.unique(np_users)
unique_items = np.unique(np_items)

n_users = unique_users.shape[0]
n_items = unique_items.shape[0]

max_item = unique_items[-1]
# Reconstruct the ratings set's user/movie indices
np_users = ratings_df.userId.values
np_users[:] -= 1 # Make users zero-indexed
# print(np_users)

# Mapping unique items down to an array 0..n_items-1
z = np.zeros(max_item+1, dtype=int)
z[unique_items] = np.arange(n_items)
movies_map = z[np_items]

np_ratings = ratings_df.rating.values
# print(np_ratings.shape[0])
ratings = np.zeros((np_ratings.shape[0], 3), dtype=object)
ratings[:, 0] = np_users
ratings[:, 1] = movies_map
ratings[:, 2] = np_ratings


X_train, X_test = train_test_split(ratings, train_size=0.8)
# print(X_train)

# Ignoring timestamp
user_train, movie_train, rating_train = zip(*X_train)
train_sparse = coo_matrix((rating_train, (user_train, movie_train)), shape=(n_users, n_items))
# print(train_sparse.shape)

user_test, movie_test, rating_test = zip(*X_test)
test_sparse = coo_matrix((rating_test, (user_test, movie_test)), shape=(n_users, n_items))
# print(test_sparse.shape)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Building the Model

In [66]:
# Network Parameters
num_input = n_items
num_hidden_1 = 10
num_hidden_2 = 5

X = tf.placeholder(tf.float64, [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}
# Encoder Hidden layer with sigmoid activation #1
en_layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(X, weights['encoder_h1']), biases['encoder_b1']))
# Encoder Hidden layer with sigmoid activation #2
encoder_op = tf.nn.sigmoid(tf.add(tf.matmul(en_layer_1, weights['encoder_h2']), biases['encoder_b2']))

# Decoder Hidden layer with sigmoid activation #1
c = tf.nn.sigmoid(tf.add(tf.matmul(encoder_op, weights['decoder_h1']), biases['decoder_b1']))
# Decoder Hidden layer with sigmoid activation #2
decoder_op  = tf.nn.sigmoid(tf.add(tf.matmul(en_layer_1, weights['decoder_h2']), biases['decoder_b2']))

y_pred = decoder_op
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)

predictions = pd.DataFrame()

# Define evaluation metrics
eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
writer.close()



### Training Phase


In [69]:
batch_size = 250
hm_epochs = 10
tot_users = train_sparse.shape[0]

train_sparse = train_sparse.tocsr()
test_sparse = test_sparse.tocsr()

init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()

sess = tf.Session()

sess.run(init)
sess.run(local_init)

In [70]:
for epoch in range(hm_epochs):
    epoch_loss = 0
    for i in range(int(tot_users/batch_size)):
        epoch_x = train_sparse[ i*batch_size : (i+1)*batch_size ]
        epoch_x = epoch_x.toarray()
        _, c = sess.run([optimizer, loss],feed_dict={X: epoch_x})
        epoch_loss += c

    if (epoch+1) % 2 == 0:
        print('MAE train', compute_MAE(sess, train_sparse, y_pred, batch_size), 
            'MAE test', compute_MAE(sess, test_sparse, y_pred, batch_size))

        print('RMSE train', np.sqrt(compute_RMSE(sess, train_sparse, y_pred, batch_size)), 
            'RMSE test', np.sqrt(compute_RMSE(sess, test_sparse, y_pred, batch_size)))

#         save_weights(sess, hidden_1_layer_vals, output_layer_vals)
print('MAE train', compute_MAE(sess, train_sparse, y_pred, batch_size), 
    'MAE test', compute_MAE(sess, test_sparse, y_pred, batch_size))

print('RMSE train', np.sqrt(compute_RMSE(sess, train_sparse, y_pred, batch_size)), 
    'RMSE test', np.sqrt(compute_RMSE(sess, test_sparse, y_pred, batch_size)))

print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)
#     save_weights(sess, hidden_1_layer_vals, output_layer_vals)

MAE train 0.5005063062963306 MAE test 0.5022091392010846
RMSE train 0.6009669974577159 RMSE test 0.602466431328224
MAE train 0.500504807364888 MAE test 0.502207601914092
RMSE train 0.6009636263599696 RMSE test 0.6024634662666482
MAE train 0.5005029569693995 MAE test 0.5022057041529595
RMSE train 0.6009594644084683 RMSE test 0.6024598056858346
MAE train 0.5005006728220612 MAE test 0.5022033614140128
RMSE train 0.600954326041148 RMSE test 0.602455286433861
MAE train 0.5004978532391244 MAE test 0.5022004694121223
RMSE train 0.6009479821416054 RMSE test 0.6024497070859958
MAE train 0.5004978532391244 MAE test 0.5022004694121223
RMSE train 0.6009479821416054 RMSE test 0.6024497070859958
Epoch 9 / 10 loss: 0.7219357490539551


In [ ]:
sess.close()